In [ ]:
!pip install fastapi pydantic uvicorn openai requests

# Start testing against HTTP endpoint

In [7]:
import requests
import json

# Base URL for the FastAPI app
base_url = "http://localhost:8000"

# Example 1: Generate a CI/CD Pipeline Configuration
def example_generate_pipeline():
    url = f"{base_url}/chat"
    payload = {
        "prompt": "Build python build pipeline"
    }
    response = requests.post(url, json=payload)
    if response.status_code == 200:
        print("Example 1 Response:")
        print(response.json())
    else:
        print("Error:", response.status_code, response.text)

# Example 2: Update a CI/CD Pipeline Configuration
def example_update_pipeline():
    url = f"{base_url}/chat"
    current_pipeline = """
    pipeline:
      stages:
        - name: Lint
          steps:
            - name: Run Flake8
              image: python:3.8
              commands:
                - pip install flake8
                - flake8 .
        - name: Test
          steps:
            - name: Run Tests
              image: python:3.8
              commands:
                - pip install -r requirements.txt
                - pytest
    """
    prompt = "Add a stage for deploying to Google Cloud Platform (GCP) using the user's credentials. Update it for me."
    payload = {
        "prompt": prompt,
        "extra_context": [
            {
                "description": "Current Pipeline",
                "content": current_pipeline
            }
        ]
    }
    response = requests.post(url, json=payload)
    if response.status_code == 200:
        print("Example 2 Response:")
        print(response.json())
    else:
        print("Error:", response.status_code, response.text)

# Example 3: Analyze Failed Pipeline Logs and Suggest a Fix
def example_fix_failed_pipeline():
    url = f"{base_url}/chat"
    logs = """
    Pipeline execution failed at stage 'Build'. Error parsing YAML configuration:
    - name: Build
      steps:
        - name: Build Image
          image: docker:latest
          commands:
            - docker build -t my-app .
            - docker push my-app:latest
          environment:
          - DOCKER_PASSWORD=${{ secrets.DOCKER_PASSWORD }}
          - DOCKER_USERNAME=${{ secrets.DOCKER_USERNAME }}
    [ERROR] Invalid YAML format at line 6, column 7: 'environment' should be a map, but found a sequence instead.
    """
    prompt = "YAML configuration error: 'environment' should be a map, but found a sequence instead."
    payload = {
        "prompt": prompt,
        "extra_context": [
            {
                "description": "Pipeline Logs",
                "content": logs
            }
        ]
    }
    response = requests.post(url, json=payload)
    if response.status_code == 200:
        print("Example 3 Response:")
        print(response.json())
    else:
        print("Error:", response.status_code, response.text)

# Run examples
if __name__ == "__main__":
    example_generate_pipeline()
    example_update_pipeline()
    example_fix_failed_pipeline()


Example 1 Response:
{'response': {'content': "Let's create a pipeline for a Python project build. Here is the plan:\n\n1. **Set up the pipeline structure**.\n2. **Add stages for linting, testing, and building**.\n3. **Use Docker as the runtime environment**.\n4. **Run the necessary scripts for linting, testing, and building**.\n\nHere is the YAML configuration for the pipeline:\n\n```yaml\npipeline:\n  stages:\n    - name: Lint\n      steps:\n        - run:\n            container: python:3.9\n            script:\n              - pip install flake8\n              - flake8 .\n\n    - name: Test\n      steps:\n        - run:\n            container: python:3.9\n            script:\n              - pip install pytest\n              - pytest\n\n    - name: Build\n      steps:\n        - run:\n            container: python:3.9\n            script:\n              - pip install -r requirements.txt\n              - python setup.py sdist bdist_wheel\n```\n\nI will generate this configuration for 